# Connect Eye-AI and Load Libraries

In [259]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [6]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
import numpy as np
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
from pathlib import Path, PurePath
import logging

#from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import DerivaML, Workflow, ExecutionConfiguration, VersionPart
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'www.eye-ai.org'
#host = 'dev.eye-ai.org' #for dev testing
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-03-10 17:20:33,698 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-10 17:20:33,699 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


# Configuration

In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-03-10 17:20:42,804 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-10 17:20:42,804 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [8]:


ml_instance.increment_dataset_version(dataset_rid='2-N93J', component= VersionPart.patch, description='Update to latest deriva-ml schema')

DatasetVersion(major=2, minor=0, patch=1)

In [9]:
source_dataset = "2-N93J"
asset_RID = ["2-C8JM"]
ml_instance = DerivaML(host, catalog_id="eye-ai")


preds_workflow = EA.add_workflow( 
    Workflow(
        name="LAC data template",
        url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/Sandbox_KB/Get_VGGPreds.ipynb",
        workflow_type="Test Workflow",
        )
    )

config = ExecutionConfiguration(
    datasets=[
        {
            "rid": source_dataset,
            "materialize": False,
            "version": ml_instance.dataset_version(source_dataset),
        }
    ],
    assets=asset_RID,
    workflow=preds_workflow,
    description="Instance of linking VGG19 predictions to patient-level data",
    )

exec = ml_instance.create_execution(config)

2025-03-10 17:27:31,538 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-10 17:27:31,539 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-03-10 17:27:32,773 - INFO - Creating new MINID for dataset 2-N93J
2025-03-10 17:27:33,041 - INFO - Downloading dataset minid for catalog: 2-N93J@2.0.1
2025-03-10 17:27:33,043 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-10 17:27:33,044 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-03-10 17:27:33,079 - INFO - Processing export config file: /tmp/tmpe6vb4xlu/download_spec.json
2025-03-10 17:27:33,081 - INFO - Requesting bdbag export

In [10]:
print(exec)

caching_dir: /home/kb_766/deriva-ml/cache
_working_dir: /home/kb_766/deriva-ml/DerivaML_working
execution_rid: 4-QAC4
workflow_rid: 4-M4TT
asset_paths: [PosixPath('/home/kb_766/deriva-ml/DerivaML_working/4-QAC4/asset/predictions_results.csv')]
configuration: datasets=[DatasetSpec(rid='2-N93J', materialize=False, version=DatasetVersion(major=2, minor=0, patch=1))] assets=['2-C8JM'] workflow='4-M4TT' description='Instance of linking VGG19 predictions to patient-level data'


# Get Pertinent Datasets

In [12]:
ds_bag = exec.datasets[0]

In [26]:
ds_bag.get_table_as_dataframe('Observation').columns

Index(['RID', 'RCT', 'RMT', 'RCB', 'RMB', 'Observation_ID', 'Subject', 'hba1c',
       'glaucoma_hx', 'visual_acuity_right', 'visual_acuity_left',
       'date_of_encounter', 'reviewed_date', 'provider', 'consultant',
       'dr_level', 'consult_id', 'assessment_and_recommendation',
       'additional_comments', 'return_time_frame',
       'referral_status_time_frame', 'Subject_hypertension',
       'Subject_insulin_dependent', 'Subject_pregnant', 'Subject_cataract',
       'Subject_maculopathy', 'Subject_other', 'Subject_image_quality', 'Age'],
      dtype='object')

In [34]:
# Function to update column names
pd.options.mode.copy_on_write = True
def updateCols(df, cols, colDict):
    df = df[cols]
    df.rename( columns = colDict, inplace = True )
    for c in set(cols).intersection( set(colDict) ): cols[cols.index(c)] = colDict.get(c)
    return df

cols = ['RID', 'Subject_ID', 'Subject_Gender', 'Subject_Ethnicity']
colDict = {'Image':'RID_Image', 'Observation':'RID_Observation', 'Subject':'RID_Subject'}


ds_bag.get_table_as_dataframe('Subject')







# Build up diagnosis DF for Optom and CNN
diags = ds_bag.get_table_as_dataframe('Image_Diagnosis')
diags = pd.merge( diags[diags['Execution'] == '2-C6E0'],
                   diags[diags['Diagnosis_Tag'] == 'Initial Diagnosis'],
                   on = 'Image', how = 'left', suffixes = ['_CNN', '_Optom'])

diags = updateCols( diags, cols, colDict )
del(cols[0])
cols[:0] = ['RID_Image', 'Diagnosis_Image_Expert', 'Diagnosis_Image_Expert_Count', 'Diagnosis_BYX', 'Diagnosis_BW', 'Diagnosis_VN', 'CDR_Expert']

# Merge onto diagnosis DF for Expert
diags = pd.merge( dxExpert, diags, on = 'RID_Image', how = 'left' )

# Link to image data
linkdDF = pd.merge( ds_bag.get_table_as_dataframe('Image'),
                  diags,
                  left_on = 'RID', right_on = 'RID_Image', 
                  how = 'right')

cols[:0] = ['Observation', 'Image_Side']
linkdDF = updateCols( linkdDF, cols, colDict )

# Link to observation data
linkdDF = pd.merge( ds_bag.get_table_as_dataframe('Observation'),
                   linkdDF,
                   left_on = 'RID', right_on = 'RID_Observation', 
                   how = 'right')

cols[:0] = ['Subject', 'date_of_encounter','hba1c', 'dr_level', 'glaucoma_hx', 'consultant', 'Subject_image_quality']  # removed site_mrn
linkdDF = updateCols( linkdDF, cols, colDict )

# Link to subject data
linkdDF = pd.merge( ds_bag.get_table_as_dataframe('Subject'),
                   linkdDF,
                   left_on = 'RID', right_on = 'RID_Subject', 
                   how = 'right')

cols[:0] = ['RID_Subject', 'Subject_Gender', 'Subject_Ethnicity']  # removed site_mrn
del(cols[ np.where( np.array(cols)=='RID_Subject' )[0][1] ]) # remove duplicated RID_Subject
linkdDF = updateCols( linkdDF, cols, colDict )

# Upload Results

In [156]:
# crete asset path
asset_type_name = "Diagnosis_Analysis"
asset_path = exec.execution_asset_path(asset_type_name)

# save assets to asset_path
linkdDF.to_csv(asset_path/'ImagesToVGG19.csv', index=False)
dxSubjectDF.to_csv(asset_path/'SubjectsToVGG19.csv', index=False)
#parityMetrics.to_csv(asset_path/'ParityMetrics.csv', index=False)

# upload assets to catalog
exec.upload_execution_outputs(clean_folder=True)

2025-02-21 11:08:53,056 - INFO - Initializing uploader: GenericUploader v1.7.6 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-02-21 11:08:53,058 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-21 11:08:53,058 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-02-21 11:08:53,096 - INFO - Scanning files in directory [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-Q6YM/execution-asset]...
2025-02-21 11:08:53,097 - INFO - Including file: [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-Q6YM/execution-asset/Diagnosis_Analysis/ImagesToVGG19.csv].
2025-02-21 11:08:53,097 - INFO - Including file: [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-Q6YM/execution-asset/Diagnosis_Analysis/SubjectsToVGG19.csv].
2025-0

{'Diagnosis_Analysis/ImagesToVGG19.csv': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_asset/6203f498f970ff62dc911033527f9bc4.ImagesToVGG19.csv:sS0V.K6SztU1SlnwwZqstYDpd4Fbdbj0', 'RID': '4-Q6ZA', 'RCT': '2025-02-21T19:08:53.720552+00:00', 'RMT': '2025-02-21T19:08:53.720552+00:00', 'RCB': 'https://auth.globus.org/6022643c-876c-4a47-bafa-5b9fac2c7782', 'RMB': 'https://auth.globus.org/6022643c-876c-4a47-bafa-5b9fac2c7782', 'Filename': 'ImagesToVGG19.csv', 'Description': None, 'Length': 230900, 'MD5': '6203f498f970ff62dc911033527f9bc4', 'Execution_Asset_Type': 'Diagnosis_Analysis'}, rid='4-Q6ZA'),
 'Diagnosis_Analysis/SubjectsToVGG19.csv': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'RID': '4-Q6YW', 'RCT': '2025-02-21T18:27:52.987298+00:00', 'RMT': '2025-02-21T18:27:52.987298+00:00', 'RCB': 'https://auth.globus.org/6022643c-876c-4a47-bafa-5b9fac2c7782', 'RMB': 'https://auth.globus.org/6022643c-876c-4a47-b